In [1]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import math

COMPARE_DESCRIPTORS = ["Proposed", "orb", "sphorb", "sift", "spoint", "alike", ]
ABLATION_DESCRIPTORS = ["Proposed","Proposed01", "Proposed1", "Proposed10", "Proposed_un", "Ltspoint", "tspoint", "Ftspoint"] 
NOT_T_DESCRIPTORS = ["spoint", "tspoint", "sphorb", "Ltspoint", "Proposed01", "Proposed03", "Proposed05", "Proposed1", "Proposed3", "Proposed", "Proposed10", "Proposed20", "Proposed_un", "Ftspoint"]
METHODS = ["", "t"]
PARAMS = ["R", "T"]
PARAMS_DICT = {"R": "Rotation", "T": "Translation"}
ALL_LOCS = ["Classroom", "Room", "Realistic", "Interior1", "Interior2", "Urban1", "Urban2", "Urban3", "Urban4"]
INDOORS = ["Classroom", "Room", "Realistic", "Interior1", "Interior2"]
OUTDOORS = ["Urban1", "Urban2", "Urban3", "Urban4"]
POSES = ["pose1", "pose2", "pose3", "pose4", "pose5"]
TIME_CATEGORYS = ["FP", "MC", "PE"]
DESCRIPTORS_DICT = {"Proposed1": "Proposed (1%)",
                    "Proposed01": "Proposed (0.1%)",
                    "Proposed03": "Proposed (0.3%)",
                    "Proposed05": "Proposed (0.5%)",
                    "Proposed3": "Proposed (3%)",
                    "Proposed": "Proposed (5%)",
                    "Proposed10": "Proposed (10%)",
                    "Proposed20": "Proposed (20%)",
                    "Proposed_un": "MNN",
                    "orb": "ORB", 
                    "sphorb": "SPHORB",
                    "sift": "SIFT",
                    "alike": "ALIKE",
                    "spoint": "SPoint",
                    "tspoint": "KNN+L",
                    "Ltspoint": "Proposed (5%)+L",
                    "Ftspoint": "FLANN"
                    }
METHODS_DICT = {"t": "T", "":""}
INDEX_CATEGORYS = ["FP_NUM", "MATCHING_NUM", "MATCHING_SCORE", "MEAN_MATCHING_ACCURCY"]


def read_csv_data(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        data = [float(row[0]) for row in reader]
    return data

In [2]:
def calc_index():
    for idx_points, num_points in enumerate([500, 1000, 10000]):
        print(num_points)
        base_path = f"results/FP_{num_points}/values"
        for tgt in ["compare", "ablation"]:
            if tgt == "compare":
                descriptors = COMPARE_DESCRIPTORS
            else:
                descriptors = ABLATION_DESCRIPTORS
            for index_category in INDEX_CATEGORYS:
                print(index_category)
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        all_time_data = []
                        for scene in ALL_LOCS:
                            file_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT/{index_category}.csv"
                            time_data = read_csv_data(file_path)
                            all_time_data.extend(time_data)
                        print(method, descriptor, np.mean(all_time_data))
    

calc_index()

500
FP_NUM
 Proposed 494.926
 orb 497.6875
t orb 497.56189903846155
 sphorb 508.6416266025641
 sift 498.80471584038696
t sift 499.57634669906844
 spoint 497.0752
 alike 498.13930148534723
t alike 488.3214
MATCHING_NUM
 Proposed 112.1828
 orb 65.59895833333333
t orb 82.0849358974359
 sphorb 79.8016826923077
 sift 31.790407093913746
t sift 52.51478331308222
 spoint 26.9468
 alike 76.70533922119631
t alike 69.7964
MATCHING_SCORE
 Proposed 0.22547484956903735
 orb 0.13135050918518068
t orb 0.1643269338840794
 sphorb 0.15631922076394345
 sift 0.06365984944633558
t sift 0.10508969385308985
 spoint 0.05421215118354766
 alike 0.15354794343314834
t alike 0.14517542262684852
MEAN_MATCHING_ACCURCY
 Proposed 0.6212823241608101
 orb 0.4465945218117177
t orb 0.52169380300867
 sphorb 0.5132336582162589
 sift 0.7001364212263743
t sift 0.7464196443733977
 spoint 0.8754895666809991
 alike 0.5340036914623307
t alike 0.5611824017973558
FP_NUM
 Proposed 494.926
 Proposed01 494.926
 Proposed1 494.926
 Propo

In [6]:
def calc_index_farm():
    for idx_points, num_points in enumerate([500, 1000, 10000]):
        print(num_points)
        base_path = f"results/FP_{num_points}/values"
        for tgt in ["compare", "ablation"]:
            if tgt == "compare":
                descriptors = COMPARE_DESCRIPTORS
            else:
                descriptors = ABLATION_DESCRIPTORS
            for index_category in INDEX_CATEGORYS:
                print(index_category)
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        all_time_data = []
                        for pose in POSES:
                            file_path = f"{base_path}/{pose}_{method}{descriptor}_5PA_GSM_wRT/{index_category}.csv"
                            time_data = read_csv_data(file_path)
                            all_time_data.extend(time_data)
                        print(method, descriptor, np.mean(all_time_data))
    

calc_index_farm()

500
MATCHING_NUM
 Proposed 149.6
 orb 130.048
t orb 109.298
 sphorb 99.712
 sift 81.506
t sift 95.21
 spoint 100.022
 alike 53.858
t alike 45.916
MATCHING_SCORE
 Proposed 0.31530799505708357
 orb 0.023971485816742966
t orb 0.030989344379597713
 sphorb 0.20811185773782653
 sift 0.07168813996707632
t sift 0.13646320098714382
 spoint 0.11070994607424202
 alike 0.036816788563950285
t alike 0.03323250709324588
MEAN_MATCHING_ACCURCY
 Proposed 0.9456911616461486
 orb 0.9719498667557259
t orb 0.9850872662847664
 sphorb 0.9447927540692214
 sift 0.9647834796023614
t sift 0.9706988978326663
 spoint 0.9893481313573548
 alike 0.9848440708479722
t alike 0.9938089773184503
MATCHING_NUM
 Proposed 149.6
 Proposed01 70.248
 Proposed1 145.062
 Proposed10 100.648
 Proposed_un 100.648
 Ltspoint 66.63
 tspoint 100.648
 Ftspoint 102.478
MATCHING_SCORE
 Proposed 0.31530799505708357
 Proposed01 0.146613235784712
 Proposed1 0.30563445066596295
 Proposed10 0.2127030786009516
 Proposed_un 0.2127030786009516
 Ltsp